In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('sample_size.csv')

In [5]:
data.head()

,customer_id,age,gender,avg_order_value,orders_last_30d,days_since_last_order,used_voucher,return_30d
0,C001,25,M,320000,2,10,1,1
1,C002,32,F,150000,1,25,0,0
2,C003,41,M,560000,3,5,1,1
3,C004,29,F,210000,1,30,0,0
4,C005,35,M,450000,2,12,1,1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            20 non-null     object
 1   age                    20 non-null     int64 
 2   gender                 20 non-null     object
 3   avg_order_value        20 non-null     int64 
 4   orders_last_30d        20 non-null     int64 
 5   days_since_last_order  20 non-null     int64 
 6   used_voucher           20 non-null     int64 
 7   return_30d             20 non-null     int64 
dtypes: int64(6), object(2)
memory usage: 1.4+ KB


In [7]:
# rule of thumb : n/p < 5 nguy hiểm : với n là số sample, p là số features

In [8]:
len(data.index) / len(data.columns)

2.5

In [9]:
# Các phương pháp giải quyết bao gồm 
# 1. feature selection / feature extraction


In [10]:
data["return_30d"].value_counts(normalize=True)

return_30d
1    0.5
0    0.5
Name: proportion, dtype: float64

In [11]:
# Dữ liệu cân bằng hoàn hảo


In [12]:
group_stats = data.groupby("return_30d").agg(
    {
        "age" : "mean",
        "gender" : "count",
        "avg_order_value" : "mean",
        "orders_last_30d" : "mean",
        "days_since_last_order" : "mean",
        "used_voucher" : "mean",
    }
)


In [13]:
group_stats

,age,gender,avg_order_value,orders_last_30d,days_since_last_order,used_voucher
return_30d,,,,,,
0,28.3,10,189000.0,1.3,28.2,0.0
1,38.3,10,600000.0,3.3,6.9,1.0


In [14]:
# Những khách hàng quay lại là những khác có độ tuổi lớn hơn ..
# Không quan trọng về giới tính -> có thể bỏ cột giới tính ra ngoài
# Giá trị order của khách hàng quay lại tương đối cao -> Nên chú ý nhiều hơn những khách hàng quay lại thay vì giữ chân 
# những khách hàng ra đi 
# Những khách hàng quay lại đang có xu hướng mua hàng rõ rệt và thích sử dụng voucher

In [15]:
#pattern mining

In [16]:
# Bảng thống kê chéo: Contingency Table
pd.crosstab(data["return_30d"], data["orders_last_30d"], normalize=True)

orders_last_30d,1,2,3,4,5
return_30d,,,,,
0,0.35,0.15,0.00,0.0,0.0
1,0.00,0.15,0.15,0.1,0.1


In [17]:
pd.crosstab(data["return_30d"], data["orders_last_30d"], normalize="index")

orders_last_30d,1,2,3,4,5
return_30d,,,,,
0,0.7,0.3,0.0,0.0,0.0
1,0.0,0.3,0.3,0.2,0.2


In [18]:
# từ hai đơn trở lên xác xuất return cao

In [19]:
data["recent_purchase"] = data["days_since_last_order"] <= 10

pd.crosstab(data["recent_purchase"], data["return_30d"], normalize="index")


return_30d,0,1
recent_purchase,,
False,0.833333,0.166667
True,0.000000,1.000000


In [20]:
pd.crosstab(data["used_voucher"], data["return_30d"], normalize="index")


return_30d,0,1
used_voucher,,
0,1.0,0.0
1,0.0,1.0


In [28]:
selected_features = [
    "avg_order_value",
    "orders_last_30d",
    "days_since_last_order"
]

data[selected_features + ["return_30d"]].corr()


,avg_order_value,orders_last_30d,days_since_last_order,return_30d
avg_order_value,1.000000,0.969945,-0.870423,0.845853
orders_last_30d,0.969945,1.000000,-0.864589,0.764719
days_since_last_order,-0.870423,-0.864589,1.000000,-0.821061
return_30d,0.845853,0.764719,-0.821061,1.000000


In [22]:
# Rule-based mining

In [23]:
rule_1 = data[
    (data["orders_last_30d"] >= 2) &
    (data["days_since_last_order"] <= 10)
]

rule_1[["customer_id", "return_30d"]]


,customer_id,return_30d
0,C001,1
2,C003,1
6,C007,1
8,C009,1
12,C013,1
14,C015,1
16,C017,1
18,C019,1


In [24]:
rule_1["return_30d"].mean()


np.float64(1.0)

In [25]:
rule_2 = data[
    (data["orders_last_30d"] == 1) &
    (data["days_since_last_order"] > 25)
]

rule_2["return_30d"].mean()


np.float64(0.0)

In [26]:
MIN_SAMPLES_PER_FEATURE = 15
needed_samples = MIN_SAMPLES_PER_FEATURE * len(selected_features)

print("Samples needed:", needed_samples)
print("Current samples:", len(data))


Samples needed: 45
Current samples: 20


In [27]:
# Segment khách hàng
data["value_segment"] = pd.cut(
    data["avg_order_value"],
    bins=[0,200000,500000,1000000],
    labels=["Low","Mid","High"]
)

pd.crosstab(data["value_segment"], data["return_30d"], normalize="index")


return_30d,0,1
value_segment,,
Low,1.0,0.0
Mid,0.5,0.5
High,0.0,1.0


In [29]:
# Giả định mô hình big data nhưng lại cần phải sample size
import pandas as pd
import numpy as np

np.random.seed(42)

N = 1_000_000

df = pd.DataFrame({
    "transaction_id": range(N),
    "customer_id": np.random.randint(1, 100_001, N),
    "amount": np.random.exponential(scale=500_000, size=N),
    "is_fraud": np.random.choice([0, 1], size=N, p=[0.9992, 0.0008]),
    "channel": np.random.choice(["POS", "Online", "ATM"], size=N)
})

df.head()


,transaction_id,customer_id,amount,is_fraud,channel
0,0,15796,441343.893634,0,POS
1,1,861,820008.782295,0,Online
2,2,76821,446082.601335,0,Online
3,3,54887,5272.051913,0,Online
4,4,6266,220185.312689,0,POS


In [30]:
df['is_fraud'].value_counts(normalize=True)

is_fraud
0    0.999187
1    0.000813
Name: proportion, dtype: float64

In [31]:
#Case–Control Sampling
fraud = df[df["is_fraud"] == 1]
non_fraud = df[df["is_fraud"] == 0].sample(
    n=len(fraud) * 10,
    random_state=42
)

balanced_sample = pd.concat([fraud, non_fraud])

balanced_sample["is_fraud"].value_counts()


is_fraud
0    8130
1     813
Name: count, dtype: int64

In [32]:
stratified_sample = (
    df
    .groupby("channel", group_keys=False)
    .apply(lambda x: x.sample(frac=0.02, random_state=42))
)

stratified_sample["channel"].value_counts(normalize=True)


C:\Users\hi\AppData\Local\Temp\ipykernel_5160\148890416.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(frac=0.02, random_state=42))


channel
Online    0.333533
POS       0.333283
ATM       0.333183
Name: proportion, dtype: float64

In [33]:
customer_sample = (
    df[["customer_id", "amount"]]
    .groupby("customer_id")
    .agg(
        total_amount=("amount", "sum"),
        txn_count=("amount", "count")
    )
    .sample(n=10_000, random_state=42)
)

customer_sample.head()


,total_amount,txn_count
customer_id,,
33969,4.259380e+06,9
22857,9.326125e+06,15
19452,7.224356e+06,14
9735,9.134065e+06,13
7132,3.171738e+06,8


In [34]:
dev_sample = df.sample(frac=0.01, random_state=42)

print("Dev samples:", len(dev_sample))


Dev samples: 10000
